In [54]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

from pyspark.sql.types import *

In [3]:
import happybase

In [4]:
connection = happybase.Connection(host='localhost', port=9090)
connection.tables()

[b'mytable', b'query2']

In [5]:
b'query2' in connection.tables()

True

In [6]:
#2019-09-12:02:http://example.com/?url=003, 1
#2019-09-12:02:http://example.com/?url=004, 3
#2019-09-12:02:http://example.com/?url=005, 4
#2019-09-12:02:http://example.com/?url=006, 10

In [46]:
connection = happybase.Connection(host='localhost', port=9090)
table = happybase.Table(b'query2',connection)
table.row('2019-09-12:02:http://example.com/?url=003')
table.row('2019-09-12:02:http://example.com/?url=004')
table.row('2019-09-12:02:http://example.com/?url=005')
int(table.row('2019-09-12:02:http://example.com/?url=006')[b'count:count'])

10

In [36]:
spark = SparkSession.builder.appName("lab6").getOrCreate() # spark session

schema = StructType([StructField("date_hour_url", StringType(), True),StructField("count_unique_users", IntegerType(), True)])

df = spark.createDataFrame(sc.emptyRDD(), schema)

In [37]:
df.show()

+-------------+------------------+
|date_hour_url|count_unique_users|
+-------------+------------------+
+-------------+------------------+



In [38]:
#newRow = spark.createDataFrame([('Alk','Dhl')])
#newRow.show()

In [39]:
#df = df.union(newRow)
#df.show()

In [41]:
listOfKeys = ['2019-09-12:02:http://example.com/?url=003',\
              '2019-09-12:02:http://example.com/?url=004',\
              '2019-09-12:02:http://example.com/?url=005',\
              '2019-09-12:02:http://example.com/?url=006']

In [48]:
for key in listOfKeys:
    connection = happybase.Connection(host='localhost', port=9090)
    table = happybase.Table(b'query2',connection)
    newRow = spark.createDataFrame([(   key   ,   int(table.row(key)[b'count:count']))])
    df = df.union(newRow)
df.show()

+--------------------+------------------+
|       date_hour_url|count_unique_users|
+--------------------+------------------+
|2019-09-12:02:htt...|                 1|
|2019-09-12:02:htt...|                 3|
|2019-09-12:02:htt...|                 4|
|2019-09-12:02:htt...|                10|
+--------------------+------------------+



In [52]:
# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("allData")
# Use SQL commands to get url events for specified hour
sqlDF = spark.sql("""
    SELECT * FROM allData
    """)

print("== explain query execution plan ==")
sqlDF.explain()    #explain

== explain query execution plan ==
== Physical Plan ==
Union
:- *(1) Project [date_hour_url#83, cast(count_unique_users#84 as bigint) AS count_unique_users#98L]
:  +- Scan ExistingRDD[date_hour_url#83,count_unique_users#84]
:- Scan ExistingRDD[_1#94,_2#95L]
:- Scan ExistingRDD[_1#101,_2#102L]
:- Scan ExistingRDD[_1#107,_2#108L]
+- Scan ExistingRDD[_1#113,_2#114L]


In [53]:
sqlDF.show()

+--------------------+------------------+
|       date_hour_url|count_unique_users|
+--------------------+------------------+
|2019-09-12:02:htt...|                 1|
|2019-09-12:02:htt...|                 3|
|2019-09-12:02:htt...|                 4|
|2019-09-12:02:htt...|                10|
+--------------------+------------------+

